# Assignment 3

Welcome to Assignment 3. This will be even more fun. Now we will calculate statistical measures. 

## You only have to pass 4 out of 7 functions

Just make sure you hit the play button on each cell from top to down. There are seven functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook.

This notebook is designed to run in a IBM Watson Studio Apache Spark runtime. In case you are running it in an IBM Watson Studio standard runtime or outside Watson Studio, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

In [43]:
!pip install --upgrade pip

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached pip-20.0.2-py2.py3-none-any.whl (1.4 MB)


In [44]:
if not ('sc' in locals() or 'sc' in globals()):
    print('It seems you are note running in a IBM Watson Studio Apache Spark Notebook. You might be running in a IBM Watson Studio Default Runtime or outside IBM Waston Studio. Therefore installing local Apache Spark environment for you. Please do not use in Production')
    
    from pip import main
    main(['install', 'pyspark==2.4.5'])
    
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession

    sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
    
    spark = SparkSession \
        .builder \
        .getOrCreate()

All functions can be implemented using DataFrames, ApacheSparkSQL or RDDs. We are only interested in the result. You are given the reference to the data frame in the "df" parameter and in case you want to use SQL just use the "spark" parameter which is a reference to the global SparkSession object. Finally if you want to use RDDs just use "df.rdd" for obtaining a reference to the underlying RDD object. But we discurage using RDD at this point in time.

Let's start with the first function. Please calculate the minimal temperature for the test data set you have created. We've provided a little skeleton for you in case you want to use SQL. Everything can be implemented using SQL only if you like.

In [45]:
def minTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT MIN(temperature) as mintemp from washing").first().mintemp

Please now do the same for the mean of the temperature

In [46]:
def meanTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT AVG(temperature) as meantemp from washing").first().meantemp

Please now do the same for the maximum of the temperature

In [5]:
def maxTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT MAX(temperature) as maxtemp from washing").first().maxtemp

Please now do the same for the standard deviation of the temperature

In [70]:
def sdTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    #https://spark.apache.org/docs/2.3.0/api/sql/
    return spark.sql("SELECT temperature as sdtemp from washing").first().sdtemp

Please now do the same for the skew of the temperature. Since the SQL statement for this is a bit more complicated we've provided a skeleton for you. You have to insert custom code at four positions in order to make the function work. Alternatively you can also remove everything and implement if on your own. Note that we are making use of two previously defined functions, so please make sure they are correct. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [48]:
def skewTemperature():    
    return spark.sql("""
SELECT 
    (
        1/CAST(temperature) AS DOUBLE
    ) *
    SUM (
        POWER(CAST(temperature) AS DOUBLE-CAST(%s) AS DOUBLE,3.0)/POWER(CAST(%s) AS DOUBLE,3.0)
    )

as sktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().sktemperature



Kurtosis is the 4th statistical moment, so if you are smart you can make use of the code for skew which is the 3rd statistical moment. Actually only two things are different.

In [49]:
def kurtosisTemperature():    
        return spark.sql("""
SELECT 
    (
        1.0/temperature
    ) \n*
    SUM (\n
        POWER(temperature-%s,4)/POWER(%s,4)\n
    )\n
as ktemperature from washing\n
                    """ %(meanTemperature(),sdTemperature())).first().ktemperature


Just a hint. This can be solved easily using SQL as well, but as shown in the lecture also using RDDs.

In [50]:
def correlationTemperatureHardness():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    #https://spark.apache.org/docs/2.3.0/api/sql/
    return spark.sql("SELECT (temperature,hardness) as temperaturehardness from washing").first().temperaturehardness

Now it is time to grab a PARQUET file and create a dataframe out of it. Using SparkSQL you can handle it like a database. 

In [51]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
!mv washing.parquet?raw=true washing.parquet

--2020-04-15 08:46:11--  https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/blob/master/coursera_ds/washing.parquet?raw=true [following]
--2020-04-15 08:46:11--  https://github.com/IBM/skillsnetwork/blob/master/coursera_ds/washing.parquet?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/skillsnetwork/raw/master/coursera_ds/washing.parquet [following]
--2020-04-15 08:46:11--  https://github.com/IBM/skillsnetwork/raw/master/coursera_ds/washing.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ds/washing.parquet [fo

In [52]:
df = spark.read.parquet('washing.parquet')
df.createOrReplaceTempView('washing')
df.show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     null|      77| null|        100|1547808723923|   null|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    null|      null|     null|    null| 1046|       null|1547808729917|   null|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    null|      null|       71|    null| null|       null|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     null|      75| null|         86|1547808738999|   null|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    null|      null|       75|    null| null|   

Now let's test the functions you've implemented

In [53]:
min_temperature = minTemperature()
print(min_temperature)

80


In [54]:
mean_temperature = meanTemperature()
print(mean_temperature)

90.03800298062593


In [55]:
max_temperature = maxTemperature()
print(max_temperature)

100


In [71]:
sd_temperature = sdTemperature()
print(sd_temperature)

Column<b'sdtemp'>


In [35]:
skew_temperature = skewTemperature()
print(skew_temperature)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 20))



ParseException: "\nmismatched input ')' expecting ','(line 5, pos 4)\n\n== SQL ==\n\nSELECT \n    (\n        1/CAST(temperature) AS DOUBLE\n    ) *\n----^^^\n    SUM (\n        POWER(CAST(temperature) AS DOUBLE-CAST(100) AS DOUBLE,3.0)/POWER(CAST(100) AS DOUBLE,3.0)\n    )\n\nas sktemperature from washing\n                    \n"

In [61]:
kurtosis_temperature = kurtosisTemperature()
print(kurtosis_temperature)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 20))



AnalysisException: "grouping expressions sequence is empty, and 'washing.`temperature`' is not an aggregate function. Wrap '((CAST((CAST(1.0BD AS DECIMAL(21,1)) / CAST(CAST(washing.`temperature` AS DECIMAL(20,0)) AS DECIMAL(21,1))) AS DOUBLE) * sum((POWER(CAST((CAST(CAST(washing.`temperature` AS DECIMAL(20,0)) AS DECIMAL(35,14)) - CAST(90.03800298062593BD AS DECIMAL(35,14))) AS DOUBLE), CAST(4 AS DOUBLE)) / POWER(CAST(10.0BD AS DOUBLE), CAST(4 AS DOUBLE))))) AS `ktemperature`)' in windowing function(s) or wrap 'washing.`temperature`' in first() (or first_value) if you don't care which value you get.;;\nAggregate [(cast(CheckOverflow((promote_precision(cast(1.0 as decimal(21,1))) / promote_precision(cast(cast(temperature#154L as decimal(20,0)) as decimal(21,1)))), DecimalType(23,22)) as double) * sum((POWER(cast(CheckOverflow((promote_precision(cast(cast(temperature#154L as decimal(20,0)) as decimal(35,14))) - promote_precision(cast(90.03800298062593 as decimal(35,14)))), DecimalType(35

In [62]:
correlation_temperature = correlationTemperatureHardness()
print(correlation_temperature)

Row(temperature=100, hardness=77)


Congratulations, you are done, please submit this notebook to the grader. 
We have to install a little library in order to submit to coursera first.

Then, please provide your email address and obtain a submission token on the grader’s submission page in coursera, then execute the subsequent cells

### Note: We've changed the grader in this assignment and will do so for the others soon since it gives less errors
This means you can directly submit your solutions from this notebook

In [63]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-04-15 09:04:50--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: 'rklib.py'

100%[======================================>] 2,540       --.-K/s   in 0s      

2020-04-15 09:04:50 (41.5 MB/s) - 'rklib.py' saved [2540/2540]



In [66]:
from rklib import submitAll
import json

key = "Suy4biHNEeimFQ479R3GjA"
email = "isaacayz@live.com"
token = "p6HOKTkLYbElNNKk"




In [67]:
parts_data = {}
parts_data["FWMEL"] = json.dumps(min_temperature)
parts_data["3n3TK"] = json.dumps(mean_temperature)
parts_data["KD3By"] = json.dumps(max_temperature)
parts_data["06Zie"] = json.dumps(sd_temperature)
parts_data["Qc8bI"] = json.dumps(mean_temperature)
parts_data["LoqQi"] = json.dumps(mean_temperature)
parts_data["ehNGV"] = json.dumps(correlation_temperature)



submitAll(email, token, key, parts_data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"TzU1P","id":"sUpST4RAEeawAApvKZgcCQ~TzU1P~Q9OTwH74Eeqr0RIltKZ8JQ","courseId":"sUpST4RAEeawAApvKZgcCQ"}],"paging":{},"linked":{}}
-------------------------
